📊 Stats to Explore for GW1
1. 🔥 Top Points Per Game(PPG) Players from 2024/25
2. 💰 Best Value Players (Points per Million)
3. 🛡️ Defenders with High Returns
4. ⚠️ Injured or Flagged Players
5. 📅 Fixture Difficulty (Team-Level)

In [4]:
import json
import pandas as pd
import os

# Get the current working directory and construct the correct path
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Use absolute path to the data file
data_file_path = '/Users/brianvu/Documents/Projects/fpl-analyser/data/bootstrap_static.json'

# Check if file exists
if os.path.exists(data_file_path):
    print(f"Found data file at: {data_file_path}")
else:
    print(f"Data file not found at: {data_file_path}")
    # Try alternative paths
    alt_path = os.path.join(current_dir, 'data', 'bootstrap_static.json')
    if os.path.exists(alt_path):
        data_file_path = alt_path
        print(f"Using alternative path: {data_file_path}")

with open(data_file_path) as f:
    data = json.load(f)

players_df = pd.DataFrame(data['elements'])
teams_df = pd.DataFrame(data['teams'])

print(f"Loaded {len(players_df)} players and {len(teams_df)} teams")

# Example: Top 15 players by most points per game
most_ppg = players_df[['web_name', 'points_per_game', 'now_cost', 'minutes']].sort_values(by='points_per_game', ascending=False).head(15)

# Example: Top 15 players by points per million
players_df['points_per_million'] = players_df['total_points'] / (players_df['now_cost'] / 10)

points_per_million = players_df[['web_name', 'total_points', 'now_cost', 'points_per_million']].sort_values(by='points_per_million', ascending=False).head(15)

# Example: Top 10 defenders by total points
# Note: 'element_type' 2 corresponds to defenders in the FPL data
defenders = players_df[players_df['element_type'] == 2] 
top_defenders = defenders[['web_name', 'total_points', 'clean_sheets', 'goals_scored', 'assists', 'bonus', 'now_cost']].sort_values(by='total_points', ascending=False).head(10)

# Example: Players with injury updates and their chance of playing this round
flagged = players_df[players_df['news'].str.len() > 0]
flagged_players = flagged[['web_name', 'news', 'chance_of_playing_this_round']]

# Fixture difficulty for each team (first 3-5 fixtures)
fixtures_url = "https://fantasy.premierleague.com/api/fixtures/"
fixtures_df = pd.read_json(fixtures_url)

# Only keep fixtures from gameweek 1 to 5
early_fixtures = fixtures_df[fixtures_df['event'] <= 5]

# Count easy fixtures (difficulty <= 2) for each team
easy_home_fixtures = early_fixtures[early_fixtures['team_h_difficulty'] <= 2]['team_h'].value_counts()
easy_away_fixtures = early_fixtures[early_fixtures['team_a_difficulty'] <= 2]['team_a'].value_counts()

easy_fixtures = (easy_home_fixtures + easy_away_fixtures).sort_values(ascending=False)

print("Analysis complete! Check the variables above for results.")

Current working directory: /Users/brianvu/Documents/Projects/fpl-analyser
Found data file at: /Users/brianvu/Documents/Projects/fpl-analyser/data/bootstrap_static.json
Loaded 674 players and 20 teams
Analysis complete! Check the variables above for results.


In [5]:
# Display the analysis results

print("🔥 TOP 15 PLAYERS BY POINTS PER GAME:")
print("=" * 50)
print(most_ppg.to_string(index=False))

print("\n\n💰 TOP 15 PLAYERS BY POINTS PER MILLION:")
print("=" * 50)
print(points_per_million.to_string(index=False))

print("\n\n🛡️ TOP 10 DEFENDERS BY TOTAL POINTS:")
print("=" * 50)
print(top_defenders.to_string(index=False))

print("\n\n⚠️ PLAYERS WITH INJURY/NEWS UPDATES:")
print("=" * 50)
if len(flagged_players) > 0:
    print(flagged_players.to_string(index=False))
else:
    print("No players currently flagged with injury concerns!")

print("\n\n📅 TEAMS WITH MOST EASY FIXTURES (GW1-5):")
print("=" * 50)
print(easy_fixtures.head(10).to_string())

🔥 TOP 15 PLAYERS BY POINTS PER GAME:
   web_name points_per_game  now_cost  minutes
    M.Salah             9.1       145     3374
     Mbeumo             6.2        80     3415
       Isak             6.2       105     2758
    Haaland             5.8       140     2736
     Palmer             5.8       105     3193
      Bowen             5.7        80     2974
       Wood             5.6        75     2958
      Cunha             5.4        80     2596
      Wissa             5.3        75     2921
       Saka             5.1       100     1724
  Luis Díaz             5.1        80     2393
       Amad             5.0        65     1898
    Watkins             4.9        90     2593
B.Fernandes             4.8        90     3017
 João Pedro             4.7        75     1946


💰 TOP 15 PLAYERS BY POINTS PER MILLION:
   web_name  total_points  now_cost  points_per_million
       Sels           150        50           30.000000
     Mbeumo           236        80           29.500000
 